# Predicting customer satisfaction at Olist - a Brazilian e-commerce platform

## DATA PREPARATION 

Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import bisect 

### OLIST DATA

In [2]:
folder_path = str(os.getcwd()) + '/'
# folder_path = 'C:/Users/zoeha/Documents/Digital Economics/Computer science project/'
customers = pd.read_csv(folder_path + 'olist_customers_dataset.csv')
geolocation =  pd.read_csv(folder_path + 'olist_geolocation_dataset.csv')
order_items =  pd.read_csv(folder_path + 'olist_order_items_dataset.csv')
order_payments =  pd.read_csv(folder_path + 'olist_order_payments_dataset.csv')
order_reviews =  pd.read_csv(folder_path + 'olist_order_reviews_dataset.csv')
order_info =  pd.read_csv(folder_path + 'olist_orders_dataset.csv')
products_info = pd.read_csv(folder_path + 'olist_products_dataset.csv')
sellers = pd.read_csv(folder_path + 'olist_sellers_dataset.csv')
product_category = pd.read_csv(folder_path + 'olist_product_category_name_translation.csv')

#### Customers

In [106]:
customers.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


Create dummy variable for customer return

In [107]:
recurrent_customers = customers[customers["customer_unique_id"].isin(customers["customer_unique_id"][customers["customer_unique_id"].duplicated()])].customer_id
customers['customer_return']=(customers["customer_id"].isin(recurrent_customers)).astype(int)
customers['customer_return'].value_counts() #Highly unbalanced but will keep for final table

0    93099
1     6342
Name: customer_return, dtype: int64

Drop unwanted columns & rename the state column to join with social-economic data later.

In [108]:
customers.drop(['customer_city', 'customer_unique_id'], axis=1, inplace=True)
customers.rename(columns={"customer_state": "state_code"}, inplace=True)

#### Geolocation

In [109]:
geolocation.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP


Drop unwanted columns & drop duplicates to keep only one lat/long pair per zip_code prefix.

In [110]:
geolocation = geolocation.drop(['geolocation_city', 'geolocation_state'], axis=1)
geolocation = geolocation.drop_duplicates(subset = ['geolocation_zip_code_prefix'],ignore_index=True)

Create 2 new data frames to join with customer data later.

In [111]:
geo_customer = geolocation.rename(columns={"geolocation_zip_code_prefix":"customer_zip_code_prefix",
                                                  "geolocation_lat":"customer_lat",
                                                  "geolocation_lng":"customer_lng"})
geo_seller = geolocation.rename(columns={"geolocation_zip_code_prefix":"seller_zip_code_prefix",
                                                  "geolocation_lat":"seller_lat",
                                                  "geolocation_lng":"seller_lng"})
geo_customer.head(3)

,customer_zip_code_prefix,customer_lat,customer_lng
0,1037,-23.545621,-46.639292
1,1046,-23.546081,-46.644820
2,1041,-23.544392,-46.639499


#### Order items
Prices are in Brazilian currency - reais.

In [112]:
order_items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


Shipping_limit_date indicates limit date for seller to handle the order over to the logistic partner. It's a step in shipping process, but it's the actual delivery time that affect customer satisfaction, so we'll drop this attribute.

In [113]:
order_items.drop('shipping_limit_date', axis=1, inplace=True)

The order_item_id represents the item quantity in each order. Due to the lack of connection between the product, order and review data sets, it's not possible to link each product in a multi-item order to its own review. Therefore, we'll only keep single-item orders.

In [114]:
order_items_restricted = order_items.loc[order_items['order_item_id'] == 1,]
order_items_restricted.drop('order_item_id', axis=1, inplace=True)
order_items_restricted.head(3)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,order_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.0,17.87


#### Order reviews

In [115]:
order_reviews.head(3)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24


Take out dupplicates for same order_id by selecting the latest review based on review_answer_timestamp, as the data is already sorted by time.

In [116]:
order_reviews.drop_duplicates(subset=['order_id'], keep='last', ignore_index=True, inplace=True)

Drop dupplicate reviews due to the issue mentioned above with multi-item orders & multi reviews.

In [117]:
order_reviews.drop_duplicates(subset=['review_id'], keep=False, ignore_index=True, inplace=True)

Create a new attribute to capture the time each customer takes to respond to survey after purchasing. Then drop the timestamp columns.

In [118]:
order_reviews['review_answer_timestamp'] = pd.to_datetime(order_reviews['review_answer_timestamp'], format='%Y/%m/%d')
order_reviews['review_creation_date'] = pd.to_datetime(order_reviews['review_creation_date'], format='%Y/%m/%d')
order_reviews['review_answer_delay'] = (order_reviews['review_answer_timestamp'] - order_reviews['review_creation_date']).dt.days

Drop the columns with text, as these are in Portugese and most of values are missing.

In [119]:
order_reviews = order_reviews.drop(['review_comment_title', 'review_comment_message', 
                                    'review_creation_date', 'review_answer_timestamp'], axis=1)
order_reviews.head(3)

,review_id,order_id,review_score,review_answer_delay
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,0
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,1
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,1


#### Order info

In [120]:
order_info.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


Create aggregated attributes to track the shipping process.

In [121]:
# changing dtype to date
order_info.loc[:,'order_purchase_timestamp'] = pd.to_datetime(order_info['order_purchase_timestamp'],
                                                              format='%Y/%m/%d').dt.date
order_info.loc[:,'order_delivered_customer_date'] = pd.to_datetime(order_info['order_delivered_customer_date'],
                                                              format='%Y/%m/%d').dt.date
order_info.loc[:,'order_estimated_delivery_date'] = pd.to_datetime(order_info['order_estimated_delivery_date'],
                                                              format='%Y/%m/%d').dt.date

# track the estimated & actual days of delivery
order_info['estimated_days_of_delivery'] = (order_info['order_estimated_delivery_date'] 
                                            - order_info['order_purchase_timestamp']).astype('timedelta64[D]')
order_info['actual_days_of_delivery'] = (order_info['order_delivered_customer_date'] 
                                            - order_info['order_purchase_timestamp']).astype('timedelta64[D]')

# track whether the delivery is late
order_info['late_delivery'] = (order_info.actual_days_of_delivery > order_info.estimated_days_of_delivery).astype(int) 

Create a year column as a key join with the socio-economic data later.

In [122]:
order_info['year'] = pd.to_datetime(order_info['order_purchase_timestamp'], format='%Y/%m/%d').dt.year

Drop rows with order_status as canceled or unavailable. Drop irrelevant columns.

In [123]:
order_info = order_info[(~order_info['order_status'].isin(['canceled', 'unavailable']))]

cols_to_drop = ['order_approved_at', 'order_delivered_carrier_date', 
               'order_delivered_customer_date', 'order_estimated_delivery_date'] 
order_info = order_info.drop(cols_to_drop, axis=1)
order_info.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,estimated_days_of_delivery,actual_days_of_delivery,late_delivery,year
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02,16.0,8.0,0,2017
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24,20.0,14.0,0,2018
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08,27.0,9.0,0,2018


#### Sellers

In [124]:
sellers.head(3)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ


Drop seller_city & seller_state.

In [125]:
sellers.drop(['seller_city', 'seller_state'], axis=1, inplace=True)

#### Product info & categories

In [126]:
products_info.head(3)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


Join product info & category to get the category names in English, then drop the Portugese category names.

In [127]:
products = pd.merge(products_info, product_category, on='product_category_name', how='left')

products.drop('product_category_name', axis=1, inplace=True)

products = products.rename(columns={"product_name_lenght": "product_name_length", 
                                              "product_description_lenght": "product_description_length",
                                             "product_category_name_english": "product_category_name"})

Assign the missing values in product_category_name as 'not_reported'.

In [128]:
products['product_category_name'] = products['product_category_name'].fillna('not_reported')
products.head(3)

,product_id,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure


Regroup the product categories & create dummy variables for this new categorical variable.

In [129]:
# product_group = pd.read_csv(folder_path + 'product_group.csv', sep = ';')

In [130]:
# # join product with the new groups we created in order to delete the category name
# products = pd.merge(products, product_group, on='product_category_name', 
#                                     how='left')
# products = products.drop('product_category_name', axis=1)

In [131]:
# # create a dummy variable for each group
# just_dummies = pd.get_dummies(products['product_category'])
# products_final = pd.concat([products, just_dummies], axis=1)
# products_final = products_final.drop('product_category', axis=1)
# products_final.head()

### IGBE DATA

#### Brazil state codes
Import state codes to join with Olist data. Drop the irrelevant rows, only keep the state names & state codes.

In [132]:
state_codes = pd.read_excel(folder_path + 'state_codes.xlsx', usecols=[0,1], names=['state_name','state_code'])

state_codes = state_codes[state_codes['state_code'].notnull()]
state_codes.head(3)

,state_name,state_code
0,Acre,AC
1,Alagoas,AL
2,Amapá,AP


#### Brazil's average household income by state
Import Brazil household income data from 2016 to 2018.

In [133]:
avg_income_2016 = pd.read_excel(folder_path + 'Tabela 2.1 UF.xls', sheet_name='2016', 
                        skiprows=[0,1,2,3,4,5], usecols=[0,1], names=['state_name', '2016'])
avg_income_2017 = pd.read_excel(folder_path + 'Tabela 2.1 UF.xls', sheet_name='2017', 
                        skiprows=[0,1,2,3,4,5], usecols=[0,1], names=['state_name', '2017'])
avg_income_2018 = pd.read_excel(folder_path + 'Tabela 2.1 UF.xls', sheet_name='2018', 
                        skiprows=[0,1,2,3,4,5], usecols=[0,1], names=['state_name', '2018'])

Clean the duplicates and rows with missing values.

In [134]:
for file in [avg_income_2016, avg_income_2017, avg_income_2018]:
    file.drop_duplicates(subset=['state_name'], keep = 'first', inplace=True)
    file.dropna(axis=0, inplace=True)
    file.reset_index(drop=True, inplace=True)

Create a table with full income data by state in three years.

In [135]:
avg_household_income = avg_income_2016.merge(avg_income_2017, on='state_name', how='left')\
                                      .merge(avg_income_2018, on='state_name', how='left')

avg_household_income = pd.melt(avg_household_income, id_vars=['state_name'], value_vars=['2016', '2017', '2018'], 
                              var_name='year', value_name='avg_income')

# create a column 'year' as a key to join later
avg_household_income['year'] = avg_household_income['year'].astype(int)

Join with state_codes & drop missing values.

In [136]:
avg_household_income = pd.merge(avg_household_income, state_codes, on='state_name', how='left')
avg_household_income.dropna(axis=0, inplace=True)
avg_household_income.head(3)

,state_name,year,avg_income,state_code
2,Rondônia,2016,969.428894,RO
4,Acre,2016,832.458072,AC
6,Amazonas,2016,799.594705,AM


#### Brazil population by state
Import Brazil population data from IGBE.

In [137]:
pop = pd.read_excel(folder_path + 'serie_2001_2018_TCU.xls', skiprows=[0,1,2,3,4,5,6], header=None, 
                    usecols=[0, 16, 17, 18], names=['state_name', '2016', '2017', '2018'], nrows=32)

Manually fix the inconsistent data type in column 2018.

In [138]:
pop['2018'] = pop['2018'].astype(str)
pop.iloc[10,3] = pop.iloc[10,3].replace(" (*)","").replace(".","")
pop.iloc[11,3] = pop.iloc[11,3].replace(" (*)","").replace(".","")
pop.iloc[14,3] = pop.iloc[14,3].replace(" (**)","").replace(".","")
pop.iloc[15,3] = pop.iloc[15,3].replace(" (**)","").replace(".","")
pop.iloc[30,3] = pop.iloc[30,3].replace(" (***)","").replace(".","")
pop.iloc[31,3] = pop.iloc[31,3].replace(" (***)","").replace(".","")
pop['2018'] = pop['2018'].astype(int)

Create a new table of population by state and year.

In [139]:
population = pd.melt(pop, id_vars=['state_name'], value_vars=['2016', '2017', '2018'],
             var_name='year', value_name='state_population')
population['year'] = population['year'].astype(int)

Join with state codes & drop missing values.

In [140]:
population = pd.merge(population, state_codes, on='state_name', how='left').drop('state_name', axis=1)
population.dropna(axis=0, inplace=True)
population.head(3)

,year,state_population,state_code
1,2016,1787279,RO
2,2016,816687,AC
3,2016,4001667,AM


### FINAL TABLE
Join all the useful tables.

In [141]:
final_df = order_items_restricted.merge(order_info, on = 'order_id', how = 'inner')
final_df = final_df.merge(products, on = 'product_id', how = 'left')
final_df = final_df.merge(order_reviews, on = 'order_id', how = 'inner')
final_df = final_df.merge(customers, on = 'customer_id', how = 'left')
final_df = final_df.merge(sellers, on = 'seller_id', how='left')
final_df = final_df.merge(geo_customer, on = 'customer_zip_code_prefix', how = 'left')
final_df = final_df.merge(geo_seller, on = 'seller_zip_code_prefix', how = 'left')
final_df = pd.merge(final_df, avg_household_income, on=['state_code', 'year'], how='left')
final_df = final_df.merge(population, on=['state_code', 'year'], how='left')

Drop more columns.

In [142]:
final_df.drop(['review_id', 'seller_id', 'state_code'], axis=1, inplace=True)

For missing values in latitude/longtitude data, fill with the lat/lng pair of the closest zip_code_prefix.

In [145]:
null_cus_lat_index = final_df[final_df['customer_lat'].isnull()].index

for i in null_cus_lat_index:
    closest_zc = min(geo_customer['customer_zip_code_prefix'],
                         key=lambda x:abs(x - final_df.loc[i, 'customer_zip_code_prefix']))
    final_df.loc[i, ['customer_lat', 'customer_lng']] = geo_customer.loc[geo_customer['customer_zip_code_prefix'] == closest_zc,
                                                                        ['customer_lat', 'customer_lng']].squeeze()

In [146]:
null_seller_lat_index = final_df[final_df['seller_lat'].isnull()].index

for i in null_seller_lat_index:
    closest_zc = min(geo_seller['seller_zip_code_prefix'],
                         key=lambda x:abs(x - final_df.loc[i, 'seller_zip_code_prefix']))
    final_df.loc[i, ['seller_lat', 'seller_lng']] = geo_seller.loc[geo_seller['seller_zip_code_prefix'] == closest_zc,
                                                                       ['seller_lat', 'seller_lng']].squeeze()

In [147]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97170 entries, 0 to 97169
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   order_id                    97170 non-null  object 
 1   product_id                  97170 non-null  object 
 2   price                       97170 non-null  float64
 3   freight_value               97170 non-null  float64
 4   customer_id                 97170 non-null  object 
 5   order_status                97170 non-null  object 
 6   order_purchase_timestamp    97170 non-null  object 
 7   estimated_days_of_delivery  97170 non-null  float64
 8   actual_days_of_delivery     95470 non-null  float64
 9   late_delivery               97170 non-null  int64  
 10  year                        97170 non-null  int64  
 11  product_name_length         95783 non-null  float64
 12  product_description_length  95783 non-null  float64
 13  product_photos_qty          957

Export to a .csv file for the exploratory analysis.

In [148]:
final_df.to_csv(r'/Users/HuyenTrang/Desktop/DATA SCIENCE PROJECT/Data sets/final_df.csv',index=False)
# df_EDA.to_csv(r'C:/Users/zoeha/Documents/Digital Economics/Computer science project/final_df.csv',index=False)